In [2]:
import ffmpeg
import cv2
import os
import random
import logging
import subprocess

# Configure Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

UPLOAD_FOLDER = "uploads"
REELS_FOLDER = "reels"
THUMBNAILS_FOLDER = "thumbnails"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(REELS_FOLDER, exist_ok=True)
os.makedirs(THUMBNAILS_FOLDER, exist_ok=True)

def extract_reels(video_path, num_reels=5, reel_duration=10):
    """Extracts short reels from an 8-minute video using FFmpeg."""
    try:
        video_path = os.path.abspath(video_path)  # Ensure correct path
        logging.info(f"Processing video: {video_path}")

        # Check if video file exists
        if not os.path.exists(video_path):
            logging.error(f"Error: Video file '{video_path}' not found!")
            return []

        # Get video duration using ffprobe
        try:
            probe = ffmpeg.probe(video_path)
            total_duration = float(probe["format"]["duration"])
        except Exception as e:
            logging.error(f"FFmpeg probe failed: {e}")
            return []

        step = total_duration // num_reels
        reel_paths = []

        for i in range(num_reels):
            start_time = int(i * step + random.randint(0, step // 2))
            output_path = os.path.abspath(f"{REELS_FOLDER}/reel_{i+1}.mp4")

            logging.info(f"Extracting reel {i+1}: Start Time = {start_time}s, Duration = {reel_duration}s")

            # Using subprocess instead of ffmpeg-python to avoid compatibility issues
            command = [
                "ffmpeg",
                "-ss", str(start_time),
                "-i", video_path,
                "-t", str(reel_duration),
                "-c:v", "libx264",
                "-c:a", "aac",
                "-strict", "experimental",
                output_path,
                "-y"  # Overwrite existing files
            ]

            process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            
            if process.returncode == 0:
                logging.info(f"Reel {i+1} saved: {output_path}")
                reel_paths.append(output_path)
            else:
                logging.error(f"FFmpeg failed for reel {i+1}: {process.stderr}")

        return reel_paths

    except Exception as e:
        logging.error(f"Error extracting reels: {e}")
        return []

def extract_thumbnails(video_path, num_thumbnails=5):
    """Extracts thumbnails from an 8-minute video using OpenCV."""
    try:
        video_path = os.path.abspath(video_path)  # Ensure correct path
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            logging.error(f"Error: Cannot open video file '{video_path}'!")
            return []

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_step = total_frames // num_thumbnails
        thumbnail_paths = []

        for i in range(num_thumbnails):
            frame_number = i * frame_step + random.randint(0, frame_step // 2)
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
            success, frame = cap.read()

            if success:
                output_path = os.path.abspath(f"{THUMBNAILS_FOLDER}/thumbnail_{i+1}.jpg")
                cv2.imwrite(output_path, frame)
                logging.info(f"Thumbnail {i+1} saved: {output_path}")
                thumbnail_paths.append(output_path)
            else:
                logging.warning(f"Failed to extract thumbnail {i+1}")

        cap.release()
        return thumbnail_paths

    except Exception as e:
        logging.error(f"Error extracting thumbnails: {e}")
        return []

# Test the function manually
if __name__ == "__main__":
    video_path = r"C:\Users\saich\OneDrive\Desktop\Social Media Short Contents\sample_video.mp4"  # Make sure this exists
    if os.path.exists(video_path):
        logging.info("Starting video processing...")
        reels = extract_reels(video_path)
        thumbnails = extract_thumbnails(video_path)
        logging.info(f"Reels: {reels}")
        logging.info(f"Thumbnails: {thumbnails}")
    else:
        logging.error("Video file not found! Check the path.")

2025-03-11 19:58:38,357 - INFO - Starting video processing...
2025-03-11 19:58:38,361 - INFO - Processing video: C:\Users\saich\OneDrive\Desktop\Social Media Short Contents\sample_video.mp4
2025-03-11 19:58:38,511 - INFO - Extracting reel 1: Start Time = 13s, Duration = 10s
2025-03-11 19:58:39,667 - INFO - Reel 1 saved: C:\Users\saich\Innomatics\reels\reel_1.mp4
2025-03-11 19:58:39,670 - INFO - Extracting reel 2: Start Time = 145s, Duration = 10s
2025-03-11 19:58:41,767 - INFO - Reel 2 saved: C:\Users\saich\Innomatics\reels\reel_2.mp4
2025-03-11 19:58:41,772 - INFO - Extracting reel 3: Start Time = 217s, Duration = 10s
2025-03-11 19:58:44,572 - INFO - Reel 3 saved: C:\Users\saich\Innomatics\reels\reel_3.mp4
2025-03-11 19:58:44,572 - INFO - Extracting reel 4: Start Time = 315s, Duration = 10s
2025-03-11 19:58:46,594 - INFO - Reel 4 saved: C:\Users\saich\Innomatics\reels\reel_4.mp4
2025-03-11 19:58:46,599 - INFO - Extracting reel 5: Start Time = 432s, Duration = 10s
2025-03-11 19:58:48,5